<h1> DS200A Computer Vision Assignment</h1>

<h2>  Part Four: Extension Activities</h2>	


TensorFlow (Optional)- Now, try using TensorFlow to categorize your images. The accuracy should be significantly higher due to the usage of nueral nets



In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
from skimage.color import rgb2gray
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from skimage.transform import resize
from sklearn.model_selection import train_test_split

def TF(X_train,y_train):

    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(105, 105, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(20))
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
    model.fit(X_train,y_train,batch_size=250,epochs=50,verbose=1,validation_split=0.2)
    return model

#Read data from file
data_file = Path("data/NB_1", "cleaned_data.hdf")
data_from_nb1 = pd.read_hdf(data_file, "starting_data")

#Tranform into grayscale(as not all images are coloured) and convert it into same size
nn_input = data_from_nb1.apply(lambda x: resize(rgb2gray(x['pictures']), (105, 105)), axis=1)
X = np.array([list(i.reshape(105,105,1)) for i in nn_input])
y = pd.get_dummies(data_from_nb1['encoding'])

#Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20, random_state =42)

#Fit the model
model_nn = TF(X_train,y_train)

scores = model_nn.evaluate(X_test, y_test)
print('Accuracy on test data: ', scores[1])

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Train on 960 samples, validate on 240 samples
Epoch 1/50
960/960 [==============================] - 17s 18ms/step - loss: 0.5989 - acc: 0.7519 - val_loss: 0.2454 - val_acc: 0.9500
Epoch 2/50
960/960 [==============================] - 15s 16ms/step - loss: 0.3373 - acc: 0.8834 - val_loss: 0.2358 - val_acc: 0.9500
Epoch 3/50
960/960 [==============================] - 15s 16ms/step - loss: 0.3207 - acc: 0.8986 - val_loss: 0.2343 - val_acc: 0.9500
Epoch 4/50
960/960 [==============================] - 15s 15ms/step - loss: 0.3019 - acc: 0.9046 - val_loss: 0.2158 - val_acc: 0.9500
Epoch 5/50
960/960 [==============================] - 15s 15ms/step - loss: 0.2958 - acc: 0.9098 - val_loss: 0.2122 - val_acc: 0.9500
Epoch 6/50
960/960 [==============================] - 15s 16ms/step - loss: 0.2883 - acc: 0.9167 - val_loss: 0.2135 - val_acc: 0.9500
Epoch 7/50
960/960 [==============================] - 16s 16ms/step - loss: 0.2748 - acc: 0.9208 - val_loss: 0.2037 - val_acc: 0.9500
Epoch 8/50
960/9

In [2]:
from skimage import io
import glob

#Read Validation Input
validation_path = '20_Validation/'
file_list= os.listdir(validation_path)
image_list = [io.imread(validation_path + animal) for animal in file_list if os.path.isfile(validation_path + animal)]
validation_df = pd.DataFrame(image_list, columns=['pictures'])

#Tranform input
val_input = validation_df.apply(lambda x: resize(rgb2gray(x['pictures']), (105, 105)), axis=1)
X_val = np.array([list(i.reshape(105,105,1)) for i in val_input])

# Predict using model
res = model_nn.predict(X_val)

#Outputting the to a csv
output_file_name = 'validation_output_neural_network'
pd.DataFrame(res).apply(lambda x: np.argmax(x), axis=1).to_csv(output_file_name)
print('Saved output to: ', output_file_name)

/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Saved output to:  validation_output_neural_network


/Users/akshaypunhani/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:52: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)
